In [1]:
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim import corpora, models, similarities
from operator import itemgetter
import nltk
import urllib3 as urllib
urllib.disable_warnings()
import xmltodict

C:\Users\manue\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import json

with open('../../material/data/news_dataset.json') as json_file:  
    data = json.load(json_file)

In [3]:
data = data['articles']

In [4]:
from datetime import datetime


#allowed_categories = ['POLITICS', 'TRAVEL', 'SPORTS', 'RELIGION', 'SCIENCE', 'TECH', 'ARTS']
allowed_categories = {'POLITICS': 0, 'ENTERTAINMENT': 0, 'HEALTHY LIVING': 0, 'TRAVEL': 0, 'BUSINESS': 0, 'SPORTS':0, 'SCIENCE': 0}

#max_per_cat = 1000

filtered_data = []
filter_date = datetime.strptime('2017-08-01', "%Y-%m-%d")

for dct in data:
    if dct['category'] in allowed_categories:
        datetime_object = datetime.strptime(dct['date'], '%Y-%m-%d')
        if dct['category'] in ['POLITICS', 'ENTERTAINMENT']:
            if datetime_object >= filter_date:
                filtered_data.append(dct)
                allowed_categories[dct['category']]+=1
        else:
            filtered_data.append(dct)
            allowed_categories[dct['category']]+=1

In [5]:
allowed_categories

{'POLITICS': 7049,
 'ENTERTAINMENT': 3058,
 'HEALTHY LIVING': 6694,
 'TRAVEL': 9887,
 'BUSINESS': 5937,
 'SPORTS': 4884,
 'SCIENCE': 2178}

In [6]:
import random

politics = []
entertainment = []
health = []
travel = []
business = []
sports = []
science = []

for dct in filtered_data:
    if dct['category'] == "POLITICS":
        politics.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "POLITICS")))
    if dct['category'] == "ENTERTAINMENT":
        entertainment.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "ENTERTAINMENT")))
    if dct['category'] == "HEALTHY LIVING":
        health.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "HEALTH")))
    if dct['category'] == "TRAVEL":
        travel.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "TRAVEL")))
    if dct['category'] == "BUSINESS":
        business.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "BUSINESS")))
    if dct['category'] == "SPORTS":
        sports.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "SPORTS")))
    if dct['category'] == "SCIENCE":
        science.append(tuple((str(dct['headline']) + '. ' + str(dct['short_description']), "SCIENCE")))
        
max_samples = 2000
politics = random.sample(politics, max_samples)
entertainment = random.sample(entertainment, max_samples)
health = random.sample(health, max_samples)
travel = random.sample(travel, max_samples)
business = random.sample(business, max_samples)
sports = random.sample(sports, max_samples)
science = random.sample(science, max_samples)

articles = politics + entertainment + health + travel + business + sports + science

In [7]:
def count_samples_per_cat(samples, cats):
    counts = {}
    for name in cats:
        counts[name] = 0
    
    for corpus, label in samples:
        counts[label]+=1
        
    return counts

In [8]:
print(count_samples_per_cat(articles, ['POLITICS', 'ENTERTAINMENT', 'HEALTH', 'TRAVEL', 'BUSINESS', 'SPORTS', 'SCIENCE']))

{'POLITICS': 2000, 'ENTERTAINMENT': 2000, 'HEALTH': 2000, 'TRAVEL': 2000, 'BUSINESS': 2000, 'SPORTS': 2000, 'SCIENCE': 2000}


In [9]:
def split_samples(samples, division): # consistent split
    
    samples_p = [(corpus, label) for (corpus, label) in samples if label == 'POLITICS']
    samples_b = [(corpus, label) for (corpus, label) in samples if label == 'ENTERTAINMENT']
    samples_t = [(corpus, label) for (corpus, label) in samples if label == 'HEALTH']
    samples_s = [(corpus, label) for (corpus, label) in samples if label == 'TRAVEL']
    samples_h = [(corpus, label) for (corpus, label) in samples if label == 'BUSINESS']
    samples_sp = [(corpus, label) for (corpus, label) in samples if label == 'SPORTS']
    samples_a = [(corpus, label) for (corpus, label) in samples if label == 'SCIENCE']
    
    train_p = samples_p[:int(division*len(samples_p))]
    test_p = samples_p[int(division*len(samples_p)):]
    
    train_b = samples_b[:int(division*len(samples_b))]
    test_b = samples_b[int(division*len(samples_b)):]
    
    train_t = samples_t[:int(division*len(samples_t))]
    test_t = samples_t[int(division*len(samples_t)):]
    
    train_s = samples_s[:int(division*len(samples_s))]
    test_s = samples_s[int(division*len(samples_s)):]
    
    train_h = samples_h[:int(division*len(samples_h))]
    test_h = samples_h[int(division*len(samples_h)):]
    
    train_sp = samples_sp[:int(division*len(samples_sp))]
    test_sp = samples_sp[int(division*len(samples_sp)):]

    train_a = samples_a[:int(division*len(samples_a))]
    test_a = samples_a[int(division*len(samples_a)):]
    
    trainset = train_p + train_b + train_t + train_s + train_h + train_sp + train_a
    testset = test_p + test_b + test_t + test_s + test_h + test_sp + test_a
    
    return trainset, testset

In [10]:
def preprocess_document(doc):
    stopset = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    tokens = wordpunct_tokenize(doc)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2]
    final = [stemmer.stem(word) for word in clean]
    return final

In [11]:
p_articles = []
for corpus, title in articles:
    p_corpus = preprocess_document(corpus)
    if len(p_corpus) > 0:
        p_articles.append(tuple((p_corpus, title)))

print(p_articles[1:3])
print(len(p_articles))

[(['legisl', 'would', 'requir', 'transpar', 'facebook', 'googl', 'polit', 'ad', 'sen', 'mark', 'warner', 'ami', 'klobuchar', 'plan', 'introduc', 'new', 'bill', 'soon', 'week', 'came', 'light', 'russian', 'interest', 'bought', 'facebook', 'ad', '2016', 'elect'], 'POLITICS'), (['american', 'defi', 'donald', 'trump', 'pro', 'daca', 'demonstr', 'erupt', 'across', 'countri', 'lo', 'angel', 'new', 'york', 'thousand', 'took', 'street', 'tuesday', 'defenddaca'], 'POLITICS')]
13999


In [12]:
print(count_samples_per_cat(articles, ['POLITICS', 'ENTERTAINMENT', 'HEALTH', 'TRAVEL', 'BUSINESS', 'SPORTS', 'SCIENCE']))

{'POLITICS': 2000, 'ENTERTAINMENT': 2000, 'HEALTH': 2000, 'TRAVEL': 2000, 'BUSINESS': 2000, 'SPORTS': 2000, 'SCIENCE': 2000}


### Feature Extraction

In [13]:
from nltk import FreqDist

dict_from_articles = [] # list of all tokens contained in the whole set of articles
for tokens, label in p_articles:
    dict_from_articles = dict_from_articles + tokens

print(len(dict_from_articles))
fdist = FreqDist(dict_from_articles) # compute frequency distribution

print(fdist)
topK = fdist.most_common(2000)

dictionary = []
for word, count in topK:
    dictionary.append(word)
    
print(len(dictionary))
print(dictionary[1:5])

232833
<FreqDist with 18390 samples and 232833 outcomes>
2000
['trump', 'year', 'one', 'time']


In [14]:
def extract_MVB_features(tokens):
    feature_vec = {}
    
    for word in dictionary:
        if word in tokens:
            feature_vec[word] = 1
        else:
            feature_vec[word] = 0
            
    return feature_vec

def extract_M_features(tokens):
    feature_vec = {}
    freqs = FreqDist(tokens)
        
    for word in dictionary:
        if word in freqs: # if word appears in the phrase
            feature_vec[word] = freqs[word]
        else:
            feature_vec[word] = 0
            
    return feature_vec

def extract_MNorm_features(tokens):
    feature_vec = {}
    freqs = FreqDist(tokens)
    div = len(tokens)
    
    for word in dictionary:
        if word in freqs: # if word appears in the phrase
            feature_vec[word] = freqs[word]
        else:
            feature_vec[word] = 0
        feature_vec[word] = round(feature_vec[word]/div,2)
            
    return feature_vec



In [15]:
import random
random.shuffle(p_articles)

featuresets_mvb = [(extract_MVB_features(corpus), label) for (corpus, label) in p_articles]
featuresets_m = [(extract_M_features(corpus), label) for (corpus, label) in p_articles]
featuresets_mn = [(extract_MNorm_features(corpus), label) for (corpus, label) in p_articles]

In [16]:
trainset_mvb, testset_mvb = split_samples(featuresets_mvb, 0.8)
trainset_m, testset_m = split_samples(featuresets_m, 0.8)
trainset_mn, testset_mn = split_samples(featuresets_mn, 0.8)

## Naive Bayes Classifier

### Multi-Variate Bernoulli Feature Set

In [17]:
bayes_classifier_mvb = nltk.NaiveBayesClassifier.train(trainset_mvb)
print("MVB: "+str(nltk.classify.accuracy(bayes_classifier_mvb, testset_mvb)))
bayes_classifier_mvb.show_most_informative_features(5)

MVB: 0.7514285714285714
Most Informative Features
                   trump = 1              POLITI : TRAVEL =    468.3 : 1.0
                  travel = 1              TRAVEL : ENTERT =    219.0 : 1.0
               scientist = 1              SCIENC : TRAVEL =    139.0 : 1.0
                   space = 1              SCIENC : SPORTS =    112.3 : 1.0
                     nfl = 1              SPORTS : ENTERT =     98.3 : 1.0


### Multinomial Feature Set

In [18]:
bayes_classifier_m = nltk.NaiveBayesClassifier.train(trainset_m)
print("M: "+str(nltk.classify.accuracy(bayes_classifier_m, testset_m)))
bayes_classifier_m.show_most_informative_features(5)

M: 0.7392857142857143
Most Informative Features
                   trump = 1              POLITI : TRAVEL =    369.0 : 1.0
                  travel = 1              TRAVEL : ENTERT =    143.7 : 1.0
               scientist = 1              SCIENC : TRAVEL =    134.3 : 1.0
                  presid = 1              POLITI : HEALTH =    115.0 : 1.0
                   trump = 2              POLITI : SCIENC =     93.7 : 1.0


### Normalized Multinomial Feature Set

In [19]:
bayes_classifier_mn = nltk.NaiveBayesClassifier.train(trainset_mn)
print("MNorm: "+str(nltk.classify.accuracy(bayes_classifier_mn, testset_mn)))
bayes_classifier_mn.show_most_informative_features(5)

MNorm: 0.5728571428571428
Most Informative Features
                   trump = 0.06           POLITI : TRAVEL =    107.0 : 1.0
                   trump = 0.05           POLITI : HEALTH =     87.0 : 1.0
                   photo = 0.05           TRAVEL : POLITI =     59.7 : 1.0
                   trump = 0.08           POLITI : SCIENC =     46.3 : 1.0
                  travel = 0.05           TRAVEL : ENTERT =     41.0 : 1.0


## Maximum Entropy Classifier

### Multi-Variate Bernoulli Feature Set

In [20]:
maxent_classifier_mvb = nltk.classify.MaxentClassifier.train(trainset_mvb, 'IIS', trace=0, max_iter=5)
print("MVB: "+str(nltk.classify.accuracy(maxent_classifier_mvb, testset_mvb)))
maxent_classifier_mvb.show_most_informative_features(5)

C:\Users\manue\Anaconda3\lib\site-packages\nltk\classify\maxent.py:1392: RuntimeWarning: overflow encountered in power
  exp_nf_delta = 2 ** nf_delta
C:\Users\manue\Anaconda3\lib\site-packages\nltk\classify\maxent.py:1394: RuntimeWarning: invalid value encountered in multiply
  sum1 = numpy.sum(exp_nf_delta * A, axis=0)
C:\Users\manue\Anaconda3\lib\site-packages\nltk\classify\maxent.py:1395: RuntimeWarning: invalid value encountered in multiply
  sum2 = numpy.sum(nf_exp_nf_delta * A, axis=0)
C:\Users\manue\Anaconda3\lib\site-packages\nltk\classify\maxent.py:1402: RuntimeWarning: invalid value encountered in true_divide
  deltas -= (ffreq_empirical - sum1) / -sum2


MVB: 0.14285714285714285
     nan new==0 and label is 'POLITICS'
     nan trump==0 and label is 'POLITICS'
     nan year==1 and label is 'POLITICS'
     nan one==0 and label is 'POLITICS'
     nan time==0 and label is 'POLITICS'


### Multinomial Feature Set

In [21]:
maxent_classifier_m = nltk.classify.MaxentClassifier.train(trainset_m, 'IIS', trace=0, max_iter=5)
print("M: "+str(nltk.classify.accuracy(maxent_classifier_m, testset_m)))
maxent_classifier_m.show_most_informative_features(5)

M: 0.14285714285714285
     nan new==0 and label is 'POLITICS'
     nan trump==0 and label is 'POLITICS'
     nan year==1 and label is 'POLITICS'
     nan one==0 and label is 'POLITICS'
     nan time==0 and label is 'POLITICS'


### Normalized Multinomial Feature Set

In [22]:
maxent_classifier_mn = nltk.classify.MaxentClassifier.train(trainset_mn, 'IIS', trace=0, max_iter=5)
print("MNorm: "+str(nltk.classify.accuracy(maxent_classifier_mn, testset_mn)))
maxent_classifier_mn.show_most_informative_features(5)

MNorm: 0.14285714285714285
     nan new==0.0 and label is 'POLITICS'
     nan trump==0.0 and label is 'POLITICS'
     nan year==0.05 and label is 'POLITICS'
     nan one==0.0 and label is 'POLITICS'
     nan time==0.0 and label is 'POLITICS'
